In [ ]:
import glob
import random
import nibabel as nb
import numpy as np
from scipy.sparse.linalg import svds, eigs
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn import image

In [ ]:
in_list = [nb.load(f) for f in glob.glob('data/cobre/fmri_*.nii.gz')]
in_mask = nb.load('data/brain_mask.nii.gz')
go = '3DMIGP'
dPCA_int = 550
dPCA_out = 100

In [ ]:
random.shuffle(in_list)

In [ ]:
mask = in_mask.get_fdata().ravel()

def demean(x):
    return x - np.mean(x, axis=0)

for i, f in enumerate(in_list):
    
    print(i, end=' ')
    
    # read data
    grot = f.get_fdata()
    grot = np.reshape(grot, [-1, grot.shape[-1]])
    grot = grot[mask!=0, :].T
    grot = demean(grot)
    
    # var-norm
    [uu, ss, vt] = svds(grot, k=30)
    vt[np.abs(vt) < (2.3 * np.std(vt))] = 0;
    stddevs = np.maximum(np.std(grot - (uu @ np.diag(ss) @ vt), axis=0), 0.001)
    grot = grot/stddevs
    
    if i == 0:
        W = demean(grot)
    else:
        # concat
        W = np.concatenate((W, demean(grot)), axis=0)
        
        # reduce W to dPCA_int eigenvectors
        if W.shape[0]-10 > dPCA_int:
            uu = eigs(W@W.T, dPCA_int)[1]
            uu = np.real(uu)
            W = uu.T @ W
        
    f.uncache()
        


In [ ]:
grot = np.zeros([mask.shape[0], dPCA_out])
grot[mask!=0, :] = W[:dPCA_out, :].T
grot = np.reshape(grot, in_list[0].shape[:3] + (dPCA_out,))

In [ ]:
nb.Nifti1Image(grot, affine=in_list[0].affine, header=in_list[0].header).to_filename('pyMIGP.nii.gz')

In [ ]:
%%bash

melodic -i pyMIGP.nii.gz \
	--mask=data/brain_mask.nii.gz \
	-d 20 \
	-v \
	--nobet \
	--disableMigp \
	--varnorm \
	-o pymigp_dim20.ica

In [ ]:
ics = nb.load('pymigp_dim20.ica/melodic_IC.nii.gz')

N = ics.shape[-1]

fig, ax = plt.subplots(int(np.ceil((N/2))),2, figsize=(12, N))

for img, ax0 in zip(image.iter_img(ics), ax.ravel()):
    coord = plotting.find_xyz_cut_coords(img, activation_threshold=1.5)
    plotting.plot_stat_map(img, cut_coords=coord, vmax=5, axes=ax0)